<a href="https://colab.research.google.com/github/loject/temp/blob/master/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
import numpy as np
import re
import glob
import os
import time, datetime
import tensorflow
import pandas

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow
print("Tensorflow version " + tensorflow.__version__)

BASE_DIR = '/content/drive/MyDrive/CNNv2'
TRAIN_FILE = '/content/drive/MyDrive/CNNv2/formated.csv'

Tensorflow version 2.8.2


In [4]:
def find_in_str(st, sub_str, pos):
  last = 0
  for i in range(pos):
    last = st.find(sub_str, last + 1)
  return last

list_files = [i for i in glob.glob(BASE_DIR + "/*.csv") if i != TRAIN_FILE] 
open(TRAIN_FILE, 'w').close()
for i in list_files[:3]:
  print(os.path.abspath(i))
  with open(i, 'r', encoding='utf-8') as f:
      text = f.read()
      text = text.split('\n')
      for j in range(len(text)):
          elem = text[j]
          if len(elem) < 1: continue
          tm = datetime.datetime.strptime(elem.split(';')[3], '%H:%M:%S').time()
          tmp = datetime.datetime.strptime(elem.split(';')[2], '%d/%m/%y') + datetime.timedelta(hours=tm.hour, minutes=tm.minute, seconds=tm.second)
          text[j] = str(tmp.timestamp()) + elem[find_in_str(elem, ';', 4):]
      
      with open(TRAIN_FILE, 'a') as f:
          for i in text:
              f.write("%s\n" % i)

/content/drive/MyDrive/CNNv2/US1.BNGO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TBIO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TRIT_200120_220925.csv


In [11]:
import os
import csv
import time, datetime

def find_in_str(st, sub_str, pos):
  last = 0
  for i in range(pos):
    last = st.find(sub_str, last + 1)
  return last
  

def process_path(file_path):
  with open(file_path) as f:
    reader = csv.reader(f, delimiter=';')
    data = [np.array(i) for i in reader if len(i) != 0]
  inp_chars = 60
  n = len(data) - inp_chars
  X = np.array([data[i:i + inp_chars] for i in range(n) if len(data[i]) != 0])
  Y = [i for i in data[inp_chars:]]
  print(len(X))
  print(len(Y))
  print(type(X))
  print(type(Y))
  return X, Y

def crutch_wrap(file_path):
  return tensorflow.numpy_function(process_path, [file_path], (tensorflow.string, tensorflow.string))

dataset = tensorflow.data.Dataset.list_files(TRAIN_FILE).map(crutch_wrap)

In [9]:
for (i, j) in dataset:
  print(type(i))
  print(type(j))
  print(i.shape)
  break

94429
94429
<class 'numpy.ndarray'>
<class 'list'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
(94429, 60, 6)


In [12]:

model = Sequential()
model.add(Input((6, 60)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(1, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(dataset, batch_size=32, epochs=100)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               24192     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 24,321
Trainable params: 24,321
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


ValueError: ignored